In [1]:
%config IPCompleter.greedy=True
import os.path
import csv
import petl as etl
from collections import OrderedDict
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, BlobBlock

In [2]:
account_name='datalake2dictas'
account_key='wlxJHcWvtVhPpL/zs6l+F1bJGKZnJ4HppZcVyh+ns32oH46E3dY/HBLau3V6um9hv+KZf/3mXEAL5nHD41X3jg=='
container_from = 'transient'
container_to = 'raw'
upload_file = '/opt/jupyter_etl_dictas/datalake/transient/beneficiario/beneficiarios_atualizado.csv'
_LOCALDIR = '/opt/jupyter_etl_dictas/datalake/transient'

In [3]:
cc_transient = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_from, credential=account_key)
cc_raw = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_to, credential=account_key)


In [6]:
_TEMP_FILE='datalake/temp'
_METADADOS='metadado/list.csv'
_SOURCE='17'
_PROC_FILES='datalake/proc'
_NM_ARQ = 'EMPRESA.csv'

### Carrega blob Empresa

#### Verifica Arquivo no Container

In [10]:
blob_list = cc_transient.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    

BENEFICIARIO.csv

EMPRESA.csv



#### Faz Download para diretório local

In [11]:
with open (f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', 'wb') as data_from :
    data_from.write(cc_transient.get_blob_client(_NM_ARQ).download_blob ().readall ())

#### Carrega dados para transformação

In [13]:
table = etl.fromcsv(f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', delimiter='|')
table

﻿pk_empresa,nome,razao_social,cnpj
00395988000135,ITAIPU,ITAIPU,00395988000135
00469585000193,FACEB,FACEB,00469585000193
02341467000120,AMAZONAS ENERGIA,AMAZONAS ENERGIA,02341467000120
02983428000127,CEMIG TELECOM,CEMIG TELECOM,02983428000127
03475839000174,ROSAL ENERGIA,ROSAL ENERGIA,03475839000174


In [15]:
#Retira os espaços vazios
def rowmapper(row):
    return [row[0].strip(),
            row[1].strip(),
            row[2].strip(),
            row[3].strip()
           ]

table1 = etl.rowmap(table, rowmapper,
                    header=['pk_empresa','nome','razao_social','cnpj'])
table1

pk_empresa,nome,razao_social,cnpj
00395988000135,ITAIPU,ITAIPU,00395988000135
00469585000193,FACEB,FACEB,00469585000193
02341467000120,AMAZONAS ENERGIA,AMAZONAS ENERGIA,02341467000120
02983428000127,CEMIG TELECOM,CEMIG TELECOM,02983428000127
03475839000174,ROSAL ENERGIA,ROSAL ENERGIA,03475839000174


#### Add fk_empresa

In [17]:
table2 = etl.addfields(table1,[('cd_empresa',_SOURCE)])
#table2
etl.tocsv(table2, f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}', delimiter='|')

In [20]:
upload_file=f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}'
#print(upload_file)
try:
    with open (upload_file, "rb") as data :
        #print(data)
        cc_raw.upload_blob(_NM_ARQ, data, overwrite = True)
finally:
    print("Tudo carregado")

Tudo carregado


#### Verifica se arquivo está no Blob

In [21]:
blob_list = cc_raw.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')

BENEFICIARIO.csv

EMPRESA.csv

